In [1]:
!pip install -q transformers datasets evaluate peft accelerate bitsandbytes nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [17]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2c5daf5e4901a4656c4fb54734741dc3649742205f2b7f116087800de73ab798
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [19]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.8 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()


Saving NLP_Problem_Solution_Dataset_Llama.csv to NLP_Problem_Solution_Dataset_Llama.csv


In [3]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files=list(uploaded.keys())[0])['train'].train_test_split(test_size=0.1)
dataset = dataset.rename_columns({"problem": "input_text", "solution": "target_text"})


Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

def preprocess_t5(example, tokenizer):
    model_input = tokenizer(example['input_text'], padding='max_length', truncation=True, max_length=512)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example['target_text'], padding='max_length', truncation=True, max_length=128)
    model_input["labels"] = labels["input_ids"]
    return model_input

t5_tokenizer = AutoTokenizer.from_pretrained("t5-small")
t5_model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

t5_tokenized = dataset.map(lambda x: preprocess_t5(x, t5_tokenizer), batched=True)

t5_args = Seq2SeqTrainingArguments(
    output_dir="./t5_finetuned",
    eval_strategy ="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs_t5",
    report_to="none"
)

t5_trainer = Seq2SeqTrainer(
    model=t5_model,
    args=t5_args,
    train_dataset=t5_tokenized["train"],
    eval_dataset=t5_tokenized["test"],
    tokenizer=t5_tokenizer
)

t5_trainer.train()


<ipython-input-21-cdc357200f8c>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  t5_trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,3.469300,3.132660
2,3.310000,3.056238
3,3.261500,3.037449


TrainOutput(global_step=3375, training_loss=3.4408382161458335, metrics={'train_runtime': 265.3441, 'train_samples_per_second': 50.855, 'train_steps_per_second': 12.719, 'total_flos': 1826302269063168.0, 'train_loss': 3.4408382161458335, 'epoch': 3.0})

In [24]:
from nltk.translate.bleu_score import corpus_bleu
from torch.utils.data import DataLoader
from nltk import word_tokenize
import nltk
import evaluate

import nltk

nltk.download('punkt')
nltk.download('punkt_tab')

rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

def generate_outputs_fast(model, tokenizer, dataset, batch_size=8, max_new_tokens=128):
    preds = []
    refs = []

    # Prepare batches
    inputs = [example["input_text"] for example in dataset]
    refs = [example["target_text"].strip() for example in dataset]

    for i in range(0, len(inputs), batch_size):
        batch_inputs = inputs[i:i+batch_size]

        tokenized = tokenizer(batch_inputs, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)
        outputs = model.generate(**tokenized, max_new_tokens=max_new_tokens)

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        preds.extend([d.strip() for d in decoded])

    return preds, refs


t5_small_preds, t5_small_refs = generate_outputs_fast(t5_model, t5_tokenizer, dataset["test"])

bleu_t5_small = corpus_bleu(
    [[word_tokenize(ref)] for ref in t5_small_refs],
    [word_tokenize(pred) for pred in t5_small_preds]
)

rouge_result = rouge.compute(predictions=t5_small_preds, references=t5_small_refs)

bertscore_result = bertscore.compute(predictions=t5_small_preds, references=t5_small_refs, lang="en")

print("5-Small Evaluation Metrics:")
print(f"- BLEU Score:   {bleu_t5_small:.4f}")
print(f"- ROUGE-1:      {rouge_result['rouge1']:.4f}")
print(f"- ROUGE-2:      {rouge_result['rouge2']:.4f}")
print(f"- ROUGE-L:      {rouge_result['rougeL']:.4f}")
print(f"- BERTScore F1: {sum(bertscore_result['f1']) / len(bertscore_result['f1']):.4f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


5-Small Evaluation Metrics:
- BLEU Score:   0.0447
- ROUGE-1:      0.2411
- ROUGE-2:      0.0597
- ROUGE-L:      0.1838
- BERTScore F1: 0.8325


In [25]:
print("\n Sample Predictions from T5-Small:")
for i in range(5):
    print(f"Problem: {dataset['test'][i]['input_text']}")
    print(f"Target:  {t5_small_refs[i]}")
    print(f"Pred:    {t5_small_preds[i]}")


 Sample Predictions from T5-Small:
Problem: The problem addressed by the research is the "deep-rooted one-pass issue" in large language model (LLM) agents, where each generated intermediate thought is plugged into the trajectory regardless of its correctness, leading to irreversible error propagation.
Target:  The proposed solution is the Generator-Assistant Stepwise Rollback (GA-Rollback) framework, which utilizes a generator to interact with the environment and an assistant to examine each action produced by the generator. The assistant triggers a rollback operation upon detection of incorrect actions. Additionally, the authors introduce two additional strategies tailored for the rollback scenario to further improve its effectiveness.
Pred:    The solution proposed is the use of a single-pass solution called a reversible error propagation (reversible error propagation) to generate intermediate thought. The solution involves: * The solution involves a single-pass solution called a re

In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
import evaluate
import torch
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')

t5_base_model_id = "t5-base"
t5_base_tokenizer = AutoTokenizer.from_pretrained(t5_base_model_id)
t5_base_model = AutoModelForSeq2SeqLM.from_pretrained(t5_base_model_id).to("cuda")

def preprocess_t5_base(example):
    inputs = t5_base_tokenizer(example['input_text'], padding='max_length', truncation=True, max_length=512)
    targets = t5_base_tokenizer(example['target_text'], padding='max_length', truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

t5_base_tokenized = dataset.map(preprocess_t5_base, batched=True)

t5_base_args = Seq2SeqTrainingArguments(
    output_dir="./t5_base_finetuned",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs_t5_base",
    report_to="none"
)

t5_base_trainer = Seq2SeqTrainer(
    model=t5_base_model,
    args=t5_base_args,
    train_dataset=t5_base_tokenized["train"],
    eval_dataset=t5_base_tokenized["test"],
    tokenizer=t5_base_tokenizer
)

t5_base_trainer.train()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

<ipython-input-27-9650c882589e>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  t5_base_trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,2.849900,2.589816
2,2.705500,2.529423
3,2.645700,2.514486


TrainOutput(global_step=3375, training_loss=2.7912185872395834, metrics={'train_runtime': 662.5041, 'train_samples_per_second': 20.368, 'train_steps_per_second': 5.094, 'total_flos': 8217277775216640.0, 'train_loss': 2.7912185872395834, 'epoch': 3.0})

In [28]:
from nltk.translate.bleu_score import corpus_bleu
from nltk import word_tokenize
import evaluate

rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

def generate_outputs_t5_base(model, tokenizer, dataset, batch_size=8, max_new_tokens=128):
    preds = []
    refs = []
    inputs = [ex["input_text"] for ex in dataset]
    refs = [ex["target_text"].strip() for ex in dataset]

    for i in range(0, len(inputs), batch_size):
        batch_inputs = inputs[i:i+batch_size]
        tokenized = tokenizer(batch_inputs, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)

        with torch.no_grad():
            outputs = model.generate(**tokenized, max_new_tokens=max_new_tokens)

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        preds.extend([d.strip() for d in decoded])

    return preds, refs

t5_base_model.eval()
t5_base_preds, t5_base_refs = generate_outputs_t5_base(t5_base_model, t5_base_tokenizer, dataset["test"])

bleu_base = corpus_bleu([[word_tokenize(r)] for r in t5_base_refs],
                        [word_tokenize(p) for p in t5_base_preds])

rouge_base = rouge.compute(predictions=t5_base_preds, references=t5_base_refs)
bertscore_base = bertscore.compute(predictions=t5_base_preds, references=t5_base_refs, lang="en")

print("T5-Base Evaluation Metrics:")
print(f"- BLEU Score:   {bleu_base:.4f}")
print(f"- ROUGE-1:      {rouge_base['rouge1']:.4f}")
print(f"- ROUGE-2:      {rouge_base['rouge2']:.4f}")
print(f"- ROUGE-L:      {rouge_base['rougeL']:.4f}")
print(f"- BERTScore F1: {sum(bertscore_base['f1']) / len(bertscore_base['f1']):.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5-Base Evaluation Metrics:
- BLEU Score:   0.0503
- ROUGE-1:      0.2522
- ROUGE-2:      0.0637
- ROUGE-L:      0.1907
- BERTScore F1: 0.8410


In [29]:
print("\n Sample Predictions from T5-Base:")
for i in range(5):
    print(f"\nProblem: {dataset['test'][i]['input_text']}")
    print(f"Target:  {t5_base_refs[i]}")
    print(f"Pred:    {t5_base_preds[i]}")


 Sample Predictions from T5-Base:

Problem: The problem addressed by the research is the "deep-rooted one-pass issue" in large language model (LLM) agents, where each generated intermediate thought is plugged into the trajectory regardless of its correctness, leading to irreversible error propagation.
Target:  The proposed solution is the Generator-Assistant Stepwise Rollback (GA-Rollback) framework, which utilizes a generator to interact with the environment and an assistant to examine each action produced by the generator. The assistant triggers a rollback operation upon detection of incorrect actions. Additionally, the authors introduce two additional strategies tailored for the rollback scenario to further improve its effectiveness.
Pred:    The proposed solution is a novel approach that leverages the inverse-reverse-correction approach to generate intermediate thought, allowing for a more efficient and efficient generation of intermediate thought. The approach involves: * Using t

In [34]:
from huggingface_hub import login

login()

In [36]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
import torch

mistral_model_id = "mistralai/Mistral-7B-Instruct-v0.2"

mistral_tokenizer = AutoTokenizer.from_pretrained(mistral_model_id)
mistral_tokenizer.pad_token = mistral_tokenizer.eos_token
mistral_model = AutoModelForCausalLM.from_pretrained(
    mistral_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

def format_prompt(example):
    return {
        "input": f"### Problem:\n{example['input_text']}\n\n### Solution:\n{example['target_text']}"
    }

mistral_dataset = dataset.map(format_prompt)
mistral_dataset = mistral_dataset.rename_column("input", "text")

def tokenize(example):
    return mistral_tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

tokenized_mistral = mistral_dataset.map(tokenize, batched=True)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM
)

mistral_model = get_peft_model(mistral_model, lora_config)

mistral_args = TrainingArguments(
    output_dir="./mistral_finetuned",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=2e-5,
    num_train_epochs=3,
    fp16=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=mistral_model,
    args=mistral_args,
    train_dataset=tokenized_mistral["train"],
    eval_dataset=tokenized_mistral["test"],
    tokenizer=mistral_tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer=mistral_tokenizer, mlm=False)
)

trainer.train()


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/4498 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

<ipython-input-36-597a11c3dc2e>:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,1.729900,1.741127
2,1.692900,1.711648
3,1.665500,1.702929


TrainOutput(global_step=6747, training_loss=1.7134006806121007, metrics={'train_runtime': 1682.3237, 'train_samples_per_second': 8.021, 'train_steps_per_second': 4.011, 'total_flos': 2.949035002681098e+17, 'train_loss': 1.7134006806121007, 'epoch': 3.0})

In [37]:
import nltk
import evaluate
from nltk.translate.bleu_score import corpus_bleu
from nltk import word_tokenize

nltk.download('punkt')

def generate_outputs_mistral(model, tokenizer, dataset, batch_size=4, max_new_tokens=128):
    model.eval()
    preds = []
    refs = []
    inputs = [f"### Problem:\n{x['input_text']}\n\n### Solution:\n" for x in dataset]
    refs = [x["target_text"].strip() for x in dataset]

    for i in range(0, len(inputs), batch_size):
        batch_inputs = inputs[i:i+batch_size]
        tokenized = tokenizer(batch_inputs, return_tensors="pt", padding=True, truncation=True, max_length=512).to(model.device)

        with torch.no_grad():
            outputs = model.generate(**tokenized, max_new_tokens=max_new_tokens)

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        batch_preds = [out.split("### Solution:")[-1].strip() for out in decoded]
        preds.extend(batch_preds)

    return preds, refs

rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

mistral_preds, mistral_refs = generate_outputs_mistral(mistral_model, mistral_tokenizer, dataset["test"])

bleu_mistral = corpus_bleu([[word_tokenize(r)] for r in mistral_refs],
                           [word_tokenize(p) for p in mistral_preds])

rouge_result = rouge.compute(predictions=mistral_preds, references=mistral_refs)

bertscore_result = bertscore.compute(predictions=mistral_preds, references=mistral_refs, lang="en")

print("Mistral Evaluation Metrics:")
print(f"- BLEU Score:   {bleu_mistral:.4f}")
print(f"- ROUGE-1:      {rouge_result['rouge1']:.4f}")
print(f"- ROUGE-2:      {rouge_result['rouge2']:.4f}")
print(f"- ROUGE-L:      {rouge_result['rougeL']:.4f}")
print(f"- BERTScore F1: {sum(bertscore_result['f1']) / len(bertscore_result['f1']):.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setti

Mistral Evaluation Metrics:
- BLEU Score:   0.0702
- ROUGE-1:      0.3180
- ROUGE-2:      0.0848
- ROUGE-L:      0.2120
- BERTScore F1: 0.8586


In [39]:
for i in range(5):
    print(f"Problem: {dataset['test'][i]['input_text']}")
    print(f"Target:  {mistral_refs[i]}")
    print(f"Mistral: {mistral_preds[i]}")


Problem: The problem addressed by the research is the "deep-rooted one-pass issue" in large language model (LLM) agents, where each generated intermediate thought is plugged into the trajectory regardless of its correctness, leading to irreversible error propagation.
Target:  The proposed solution is the Generator-Assistant Stepwise Rollback (GA-Rollback) framework, which utilizes a generator to interact with the environment and an assistant to examine each action produced by the generator. The assistant triggers a rollback operation upon detection of incorrect actions. Additionally, the authors introduce two additional strategies tailored for the rollback scenario to further improve its effectiveness.
Mistral: The proposed solution is the "Thoughtful Agent" framework, which introduces a novel "thoughtful" agent that can dynamically reconsider and revise its intermediate thoughts. The framework consists of three key components:

1. A novel "thoughtful" agent that can dynamically recons